In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
import time
import warnings
import statsmodels.api as sm
import datetime
import math
from scipy.ndimage.interpolation import shift
import shapely.wkt
from shapely.validation import explain_validity
from shapely.validation import make_valid
import xarray as xr

warnings.filterwarnings('ignore')
import seaborn as sns


from joblib import Parallel, delayed
import multiprocessing

## MERRA2 GRID

In [6]:
path_merra = '/data2/lthapa/2020/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20200801.nc4'
dat_merra = xr.open_dataset(path_merra)

#get the lat/lon of the grid
lat = dat_merra['lat'].values #37 elements, rows, y
lon = dat_merra['lon'].values #39 elements, columns, x
print(lat)
print(lon)

#drop the outer ring of lat/lons
lat_sub = lat[1:len(lat)-1] #35 elements
lon_sub = lon[1:len(lon)-1] #37 elements

#calculate the grid centers
lon_centers, lat_centers = np.meshgrid(lon_sub, lat_sub) #these are 35x37

#calculate the grid corners
lon, lat = np.meshgrid(lon,lat) #these are 37x39 
lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon) #these are 36x38

#build and save the grid
#build_grid_gdf(lat_corners, lon_corners, lat_centers, lon_centers, 'MERRA_GRID')
build_grid_netcdf(lat_corners, lon_corners, lat_centers, lon_centers, 'MERRA_GRID')

[31.  31.5 32.  32.5 33.  33.5 34.  34.5 35.  35.5 36.  36.5 37.  37.5
 38.  38.5 39.  39.5 40.  40.5 41.  41.5 42.  42.5 43.  43.5 44.  44.5
 45.  45.5 46.  46.5 47.  47.5 48.  48.5 49. ]
[-125.    -124.375 -123.75  -123.125 -122.5   -121.875 -121.25  -120.625
 -120.    -119.375 -118.75  -118.125 -117.5   -116.875 -116.25  -115.625
 -115.    -114.375 -113.75  -113.125 -112.5   -111.875 -111.25  -110.625
 -110.    -109.375 -108.75  -108.125 -107.5   -106.875 -106.25  -105.625
 -105.    -104.375 -103.75  -103.125 -102.5   -101.875 -101.25 ]
35 37
36 38
<xarray.Dataset>
Dimensions:   (cols_cor: 38, cols_ctr: 37, rows_cor: 36, rows_ctr: 35)
Coordinates:
  * rows_ctr  (rows_ctr) int64 0 1 2 3 4 5 6 7 8 ... 26 27 28 29 30 31 32 33 34
  * cols_ctr  (cols_ctr) int64 0 1 2 3 4 5 6 7 8 ... 28 29 30 31 32 33 34 35 36
  * rows_cor  (rows_cor) int64 0 1 2 3 4 5 6 7 8 ... 27 28 29 30 31 32 33 34 35
  * cols_cor  (cols_cor) int64 0 1 2 3 4 5 6 7 8 ... 29 30 31 32 33 34 35 36 37
Data variables:
    L

## RAVE

In [8]:
os.system('ncrename -v Latitude,lat -v Longitude,lon /data2/lthapa/2020/Apr2020_to_Oct2020/Hourly_Emissions_FV3_13km_202004070000_202004072300.nc /data2/lthapa/2020/Apr2020_to_Oct2020/Hourly_Emissions_FV3_13km_202004070000_202004072300xr.nc')


0

In [9]:
path_rave = '/data2/lthapa/2020/Apr2020_to_Oct2020/Hourly_Emissions_FV3_13km_202004070000_202004072300xr.nc'
dat_rave = xr.open_dataset(path_rave)
print(dat_rave)
#get the lat/lon of the grid, these are the edges/corners
lat_corners = dat_rave['lat'].values #233 x 397
lon_corners = dat_rave['lon'].values - 360 #233 x 397
print(dat_rave['Latitude'].values)
#calculate the centers
lat_centers, lon_centers = calculate_grid_cell_corners(lat_corners, lon_corners)

#build and save the grid
#build_grid_gdf(lat_corners, lon_corners, lat_centers, lon_centers, 'RAVE_GRID')
build_grid_netcdf(lat_corners, lon_corners, lat_centers, lon_centers, 'RAVE_GRID')

<xarray.Dataset>
Dimensions:   (Latitude: 233, Longitude: 397, Time: 24, xFRP: 396, yFRP: 232)
Dimensions without coordinates: Latitude, Longitude, Time, xFRP, yFRP
Data variables: (12/14)
    lat       (Latitude, Longitude) float32 ...
    lon       (Latitude, Longitude) float32 ...
    time      (Time) datetime64[ns] ...
    area      (yFRP, xFRP) float32 ...
    Mean_FRP  (yFRP, xFRP, Time) float32 ...
    FRE       (yFRP, xFRP, Time) float32 ...
    ...        ...
    SO2       (yFRP, xFRP, Time) float32 ...
    OC        (yFRP, xFRP, Time) float32 ...
    BC        (yFRP, xFRP, Time) float32 ...
    PM2.5     (yFRP, xFRP, Time) float32 ...
    NOx       (yFRP, xFRP, Time) float32 ...
    NH3       (yFRP, xFRP, Time) float32 ...
Attributes:
    PRODUCT_ALGORITHM_VERSION:        Beta
    TIME_RANGE:                       24 hours
    RangeBeginningDate(YYYY-MM-DD):   2020-04-07
    RangeBeginningTime(UTC-hour):     00
    RangeEndingDate(YYYY-MM-DD):      2020-04-07
    RangeEndingT

## ESI

In [15]:
path_esi = '/data2/lthapa/DFPPM_4WK_2020323.nc'
dat_esi = xr.open_dataset(path_esi)

#get the lat/lon of the grid
lat = dat_esi['lat'].values #360 elements, rows, y
lon = dat_esi['lon'].values #480 elements, columns, x


#drop the outer ring of lat/lons
lat_sub = lat[1:len(lat)-1] #358 elements
lon_sub = lon[1:len(lon)-1] #478 elements

#calculate the grid centers
lon_centers, lat_centers = np.meshgrid(lon_sub, lat_sub) #these are 358x478

#calculate the grid corners
lon, lat = np.meshgrid(lon,lat) #these are 360x480
lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon) #these are 359x479
print(lon_corners.shape)

#build and save the grid
build_grid_gdf(lat_corners, lon_corners, lat_centers, lon_centers, 'ESI_GRID')

(359, 479)
358 478
           lat      lon    row    col  \
0       31.075 -124.925    0.0    0.0   
1       31.075 -124.875    0.0    1.0   
2       31.075 -124.825    0.0    2.0   
3       31.075 -124.775    0.0    3.0   
4       31.075 -124.725    0.0    4.0   
...        ...      ...    ...    ...   
171119  48.925 -101.275  357.0  473.0   
171120  48.925 -101.225  357.0  474.0   
171121  48.925 -101.175  357.0  475.0   
171122  48.925 -101.125  357.0  476.0   
171123  48.925 -101.075  357.0  477.0   

                                                 geometry  
0       POLYGON ((-124.95000 31.05000, -124.95000 31.1...  
1       POLYGON ((-124.90000 31.05000, -124.90000 31.1...  
2       POLYGON ((-124.85000 31.05000, -124.85000 31.1...  
3       POLYGON ((-124.80000 31.05000, -124.80000 31.1...  
4       POLYGON ((-124.75000 31.05000, -124.75000 31.1...  
...                                                   ...  
171119  POLYGON ((-101.30000 48.90000, -101.30000 48.9...  
171120  

## PWS Grid

In [10]:
path_pws = '/data2/lthapa/PWS_6_jan_2021.nc'
dat_pws = xr.open_dataset(path_pws)

#get the lat/lon of the grid
lat = dat_pws['lat'].values #645 elements, rows, y
lon = dat_pws['lon'].values #870 elements, columns, x

#drop the outer ring of lat/lons
lat_sub = lat[1:len(lat)-1] #643 elements
lon_sub = lon[1:len(lon)-1] #868 elements


#calculate the grid centers
lon_centers, lat_centers = np.meshgrid(lon_sub, lat_sub) #these are 643x868

#calculate the grid corners
lon, lat = np.meshgrid(lon,lat) #these are 645x870
lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon) #these are 644x869

#build and save the grid
build_grid_gdf(lat_corners, lon_corners, lat_centers, lon_centers, 'PWS_GRID')


643 868
              lat         lon    row    col  \
0       25.889446 -124.704128    0.0    0.0   
1       25.889446 -124.668195    0.0    1.0   
2       25.889446 -124.632263    0.0    2.0   
3       25.889446 -124.596330    0.0    3.0   
4       25.889446 -124.560397    0.0    4.0   
...           ...         ...    ...    ...   
558119  48.958183  -93.694284  642.0  863.0   
558120  48.958183  -93.658352  642.0  864.0   
558121  48.958183  -93.622419  642.0  865.0   
558122  48.958183  -93.586486  642.0  866.0   
558123  48.958183  -93.550554  642.0  867.0   

                                                 geometry  
0       POLYGON ((-124.72209 25.87148, -124.72209 25.9...  
1       POLYGON ((-124.68616 25.87148, -124.68616 25.9...  
2       POLYGON ((-124.65023 25.87148, -124.65023 25.9...  
3       POLYGON ((-124.61430 25.87148, -124.61430 25.9...  
4       POLYGON ((-124.57836 25.87148, -124.57836 25.9...  
...                                                   ...  
558119 

## HRRR Grid

In [8]:
path_hrrr = '/data2/lthapa/ML_daily/pygraf/Processed_HRRR_2020091800.nc'
dat_hrrr = xr.open_dataset(path_hrrr)

lat_centers = dat_hrrr['grid_latt'].values #1057x1797
lon_centers = dat_hrrr['grid_lont'].values #1057x1797

#drop the outer ring of lat/lons
lat_centers = lat_centers[1:lat_centers.shape[0],1:lat_centers.shape[1]] #1056x1796
lon_centers = lon_centers[1:lon_centers.shape[0],1:lon_centers.shape[1]] #1056x1796

#load in the corners
lat_corners = dat_hrrr['grid_lat'].values #1057x1797
lon_corners = dat_hrrr['grid_lon'].values #1057x1797

#build_grid_netcdf(lat_corners, lon_corners, lat_centers, lon_centers, 'HRRR_GRID')


nrows = lat_centers.shape[0]
ncols = lat_centers.shape[1]

print(nrows, ncols)

row_ind = range(nrows)
col_ind =range(ncols)

print(multiprocessing.cpu_count())

tic = time.time()
results = Parallel(n_jobs=6)(delayed(build_one_gridcell)(lat_corners, lon_corners, lat_centers, lon_centers,i,j) for i in row_ind for j in col_ind)
toc = time.time()
print(toc-tic)

df_grid=gpd.GeoDataFrame(results)
df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
df_grid.set_geometry(col='geometry', inplace=True) #assign a column to represent geometry

print(df_grid)
#tic =time.time()
#df_grid.to_file('HRRR_GRID.geojson', driver='GeoJSON')
#toc=time.time()
print(toc-tic)
#df_size = len(row_ind)*len(col_ind)
#df_grid = gpd.GeoDataFrame({'lat': np.zeros(df_size), 
#                                'lon': np.zeros(df_size),
#                                'row':np.zeros(df_size),
#                                'col':np.zeros(df_size),
#                                'geometry':np.zeros(df_size)})
#df_grid = Parallel(n_jobs=2)(delayed(store_grid)(df_grid,results,kk) for kk in range(df_size))
#print(df_grid)
#build_one_gridcell(lat_corners, lon_corners, lat_centers, lon_centers,0,0)


1056 1796
64
70.05691599845886
               lat         lon   row   col  \
0        21.201847 -122.681160     0     0   
1        21.208826 -122.654472     0     1   
2        21.215797 -122.627777     0     2   
3        21.222763 -122.601082     0     3   
4        21.229721 -122.574379     0     4   
...            ...         ...   ...   ...   
1896571  47.869011  -61.115360  1055  1791   
1896572  47.858761  -61.078770  1055  1792   
1896573  47.848503  -61.042194  1055  1793   
1896574  47.838234  -61.005627  1055  1794   
1896575  47.827953  -60.969074  1055  1795   

                                                  geometry  
0        POLYGON ((-122.69075 21.18592, -122.69825 21.2...  
1        POLYGON ((-122.66408 21.19290, -122.67155 21.2...  
2        POLYGON ((-122.63738 21.19987, -122.64487 21.2...  
3        POLYGON ((-122.61069 21.20684, -122.61816 21.2...  
4        POLYGON ((-122.58400 21.21380, -122.59146 21.2...  
...                                               

In [63]:
def store_grid(grid,res, k):
    grid.iloc[k,:] = res[k]
    return grid

In [10]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices
def build_one_gridcell(LAT_COR, LON_COR, LAT_CTR, LON_CTR, ii,jj):
    #print(ii,jj,count)
    #print(LAT_CTR[ii,jj], LON_CTR[ii,jj]) #ctr
    sw = (LON_COR[ii, jj],LAT_COR[ii, jj]) #SW
    se =(LON_COR[ii, jj+1],LAT_COR[ii, jj+1]) #SE
    nw = (LON_COR[ii+1, jj],LAT_COR[ii+1, jj]) #NW
    ne = (LON_COR[ii+1, jj+1],LAT_COR[ii+1, jj+1]) #NE
            
    poly_cell = Polygon([sw,nw,ne,se])
    
    return LAT_CTR[ii,jj], LON_CTR[ii,jj],ii,jj,poly_cell

## CPC FWI Grid

In [23]:
path_cpc = '../FWI.CPC.Daily.WESTUS.20200801.nc'
dat_cpc = xr.open_dataset(path_cpc)

#get the lat/lon of the grid
lat = dat_cpc['lat'].values #36 elements, rows, y
lon = dat_cpc['lon'].values #48 elements, columns, x

#drop the outer ring of lat/lons
lat_sub = lat[1:len(lat)-1] #34 elements
lon_sub = lon[1:len(lon)-1] #46 elements

#calculate the grid centers
lon_centers, lat_centers = np.meshgrid(lon_sub, lat_sub) #these are 34x46

#calculate the grid corners
lon, lat = np.meshgrid(lon,lat) #these are 36x48
lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon) #these are 35x47

nrows = lat_centers.shape[0]
ncols = lat_centers.shape[1]

print(nrows, ncols)

row_ind = range(nrows)
col_ind =range(ncols)

print(multiprocessing.cpu_count())

tic = time.time()
results = Parallel(n_jobs=6)(delayed(build_one_gridcell)(lat_corners, lon_corners, lat_centers, lon_centers,i,j) for i in row_ind for j in col_ind)
toc = time.time()
print(toc-tic)

df_grid=gpd.GeoDataFrame(results)
df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
df_grid.set_geometry(col='geometry', inplace=True) #assign a column to represent geometry

print(df_grid)
tic =time.time()
df_grid.to_file('CPC_FWI_GRID.geojson', driver='GeoJSON')
toc=time.time()
print(toc-tic)

34 46
64
0.23758506774902344
        lat     lon  row  col  \
0     31.75 -124.25    0    0   
1     31.75 -123.75    0    1   
2     31.75 -123.25    0    2   
3     31.75 -122.75    0    3   
4     31.75 -122.25    0    4   
...     ...     ...  ...  ...   
1559  48.25 -103.75   33   41   
1560  48.25 -103.25   33   42   
1561  48.25 -102.75   33   43   
1562  48.25 -102.25   33   44   
1563  48.25 -101.75   33   45   

                                               geometry  
0     POLYGON ((-124.50000 31.50000, -124.50000 32.0...  
1     POLYGON ((-124.00000 31.50000, -124.00000 32.0...  
2     POLYGON ((-123.50000 31.50000, -123.50000 32.0...  
3     POLYGON ((-123.00000 31.50000, -123.00000 32.0...  
4     POLYGON ((-122.50000 31.50000, -122.50000 32.0...  
...                                                 ...  
1559  POLYGON ((-104.00000 48.00000, -104.00000 48.5...  
1560  POLYGON ((-103.50000 48.00000, -103.50000 48.5...  
1561  POLYGON ((-103.00000 48.00000, -103.00000 48.5

## IMERG FWI Grid

In [33]:
path_imerg = '../FWI.IMERG.FINAL.v6.Daily.WESTUS.20200801.nc'
dat_imerg = xr.open_dataset(path_imerg)

#get the lat/lon of the grid
lat = dat_imerg['lat'].values #180 elements, rows, y
lon = dat_imerg['lon'].values #240 elements, columns, x

#drop the outer ring of lat/lons
lat_sub = lat[1:len(lat)-1] #178 elements
lon_sub = lon[1:len(lon)-1] #238 elements

#calculate the grid centers
lon_centers, lat_centers = np.meshgrid(lon_sub, lat_sub) #these are 178x238

#calculate the grid corners
lon, lat = np.meshgrid(lon,lat) #these are 36x48
lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon) #these are 179x239

nrows = lat_centers.shape[0]
ncols = lat_centers.shape[1]



row_ind = range(nrows)
col_ind =range(ncols)

print(multiprocessing.cpu_count())

tic = time.time()
results = Parallel(n_jobs=6)(delayed(build_one_gridcell)(lat_corners, lon_corners, lat_centers, lon_centers,i,j) for i in row_ind for j in col_ind)
toc = time.time()
print(toc-tic)

df_grid=gpd.GeoDataFrame(results)
df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
df_grid.set_geometry(col='geometry', inplace=True) #assign a column to represent geometry

print(df_grid)
tic =time.time()
df_grid.to_file('IMERG_FWI_GRID.geojson', driver='GeoJSON')
toc=time.time()
print(toc-tic)

64
2.5746617317199707
             lat         lon  row  col  \
0      31.149998 -124.849991    0    0   
1      31.149998 -124.750000    0    1   
2      31.149998 -124.649994    0    2   
3      31.149998 -124.549995    0    3   
4      31.149998 -124.449997    0    4   
...          ...         ...  ...  ...   
42359  48.850002 -101.549995  177  233   
42360  48.850002 -101.449997  177  234   
42361  48.850002 -101.349998  177  235   
42362  48.850002 -101.249992  177  236   
42363  48.850002 -101.149994  177  237   

                                                geometry  
0      POLYGON ((-124.89999 31.10000, -124.89999 31.2...  
1      POLYGON ((-124.80000 31.10000, -124.80000 31.2...  
2      POLYGON ((-124.70000 31.10000, -124.70000 31.2...  
3      POLYGON ((-124.59999 31.10000, -124.59999 31.2...  
4      POLYGON ((-124.50000 31.10000, -124.50000 31.2...  
...                                                  ...  
42359  POLYGON ((-101.59999 48.80000, -101.59999 48.9...  
4

## Helper Function

In [2]:
def calculate_grid_cell_corners(LAT, LON):
    #we will assume the very edges of the polygons don't touch the boundary of the domain
    lat_corners = (LAT[0:(LAT.shape[0]-1),  0:(LAT.shape[1])-1] + LAT[1:(LAT.shape[0]), 1:(LAT.shape[1])])/2
    lon_corners = (LON[0:(LAT.shape[0]-1),  0:(LAT.shape[1])-1] + LON[1:(LAT.shape[0]), 1:(LAT.shape[1])])/2
    return lat_corners, lon_corners

In [3]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices
def build_one_gridcell(LAT_COR, LON_COR, LAT_CTR, LON_CTR, ii,jj):
    #print(ii,jj,count)
    #print(LAT_CTR[ii,jj], LON_CTR[ii,jj]) #ctr
    sw = (LON_COR[ii, jj],LAT_COR[ii, jj]) #SW
    se =(LON_COR[ii, jj+1],LAT_COR[ii, jj+1]) #SE
    nw = (LON_COR[ii+1, jj],LAT_COR[ii+1, jj]) #NW
    ne = (LON_COR[ii+1, jj+1],LAT_COR[ii+1, jj+1]) #NE
            
    poly_cell = Polygon([sw,nw,ne,se])
    
    return LAT_CTR[ii,jj], LON_CTR[ii,jj],ii,jj,poly_cell

In [4]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices
def build_grid_gdf(LAT_COR, LON_COR, LAT_CTR, LON_CTR, filename):
    #loop over the centers
    nrows = LAT_CTR.shape[0]
    ncols = LAT_CTR.shape[1]
    print(nrows, ncols)

    #preallocate the dataframe
    df_size = nrows*ncols
    df_grid = gpd.GeoDataFrame({'lat': np.zeros(df_size), 
                                'lon': np.zeros(df_size),
                                'row':np.zeros(df_size),
                                'col':np.zeros(df_size),
                                'geometry':np.zeros(df_size)})

    count=0
    for ii in range(nrows):
        for jj in range(ncols):
            #print(ii,jj,count)
            #print(LAT_CTR[ii,jj], LON_CTR[ii,jj]) #ctr
            sw = (LON_COR[ii, jj],LAT_COR[ii, jj]) #SW
            se =(LON_COR[ii, jj+1],LAT_COR[ii, jj+1]) #SE
            nw = (LON_COR[ii+1, jj],LAT_COR[ii+1, jj]) #NW
            ne = (LON_COR[ii+1, jj+1],LAT_COR[ii+1, jj+1]) #NE
            
            poly_cell = Polygon([sw,nw,ne,se])
            df_grid.iloc[count,:] = [LAT_CTR[ii,jj], LON_CTR[ii,jj],ii,jj,poly_cell]

            
            count=count+1
    print(df_grid)
    df_grid.set_geometry(col='geometry', inplace=True)
    df_grid.to_file(filename+'.geojson', driver='GeoJSON')

In [5]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices
def build_grid_netcdf(LAT_COR, LON_COR, LAT_CTR, LON_CTR, filename):
    #loop over the centers
    nrows_center = LAT_CTR.shape[0]
    ncols_center = LAT_CTR.shape[1]
    print(nrows_center, ncols_center)

    nrows_corner = LAT_COR.shape[0]
    ncols_corner = LAT_COR.shape[1]
    print(nrows_corner,ncols_corner)
    
    rows_ctr = np.arange(nrows_center)
    cols_ctr = np.arange(ncols_center)
    rows_cor = np.arange(nrows_corner)
    cols_cor = np.arange(ncols_corner)

    
    dat_grid = xr.Dataset(
        data_vars = dict(
            LAT_CTR=(['rows_ctr','cols_ctr'],LAT_CTR),
            LON_CTR=(['rows_ctr','cols_ctr'],LON_CTR),
            LAT_COR=(['rows_cor','cols_cor'],LAT_COR),
            LON_COR=(['rows_cor','cols_cor'],LON_COR),
        ),
        
        coords = dict(
            rows_ctr =(['rows_ctr'],rows_ctr),
            cols_ctr =(['cols_ctr'],cols_ctr),
            rows_cor =(['rows_cor'],rows_cor),
            cols_cor =(['cols_cor'],cols_cor),
        
        )
        
    )
    print(dat_grid)
    dat_grid.to_netcdf(filename+'.nc')